In [44]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
import os
from models.binarized_modules import binarized
import math
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
# from binarized_modules import  BinarizeLinear,BinarizeConv2d

In [45]:
import sys
sys.path.append('/shares/bulk/earapidis/dev/Fast-Crossbar-Sim/python')
# sys.path.append('/home/earapidis/Fast-Crossbar-Sim/python')
from crossbar import VectorSim, ParallelSim, _task
from tqdm import tqdm

In [46]:
os.getcwd()

'/shares/bulk/earapidis/dev/BinarizedNN'

In [47]:
cuda = False

In [48]:
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

In [49]:
test_batch_size=1

In [50]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

In [51]:
# from mnist_bnn import Net
from models.lenet_5 import BinarizedLeNet5_BN as Net

model = Net()
if cuda:
    torch.cuda.set_device(0)
    model.cuda()


In [52]:
# model_path = os.path.join(models_path,f"epoch_7.pth")
model_idx = 4
models_path = os.path.abspath(f"/shares/bulk/earapidis/dev/BinarizedNN/saved_models/lenet_5/model_{model_idx}")
# models_path = os.path.abspath(f"/home/earapidis/BinarizedNN/saved_models/lenet_5/model_{model_idx}")
model_path = os.path.join(models_path,f"epoch_2.pth")
# model_path = os.path.join(models_path,f"best.pth")
model = Net()
model.load_state_dict(torch.load(model_path))
if cuda:
    torch.cuda.set_device(0)
    model.cuda()

In [53]:
model.eval()

BinarizedLeNet5_BN(
  (conv1): BinarizeConv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh1): Hardtanh(min_val=-1.0, max_val=1.0)
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): BinarizeConv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh2): Hardtanh(min_val=-1.0, max_val=1.0)
  (pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc1): BinarizeLinear(in_features=256, out_features=120, bias=True)
  (bn_fc1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh3): Hardtanh(min_val=-1.0, max_val=1.0)
  (fc2): BinarizeLinear(in_features=120, out_features=84, bias=True)
  (bn_fc2): BatchNorm1d(84, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh4): Hardtanh(min_val=-1.0, max_val=1.0)
  (fc3): BinarizeLinear(in_features=84

In [54]:
images, labels = next(iter(test_loader))
print(f"image shape: {images.shape}")

image shape: torch.Size([1, 1, 28, 28])


In [55]:

x=model.conv1(images)
x=model.bn1(x)
x=model.htanh1(x)
x=model.pool1(x)
x = x.detach()

In [56]:
# inputs = images
inputs = x
inputs = binarized(inputs)
inputs.shape

torch.Size([1, 6, 12, 12])

In [58]:
filters = model.conv2.weight.data
# filters = model.conv1.weight.data
filters = binarized(filters)
filters.shape

torch.Size([16, 6, 5, 5])

In [59]:

COUT, CIN, Kh, Kw = filters.shape
N, CIN, Hi,Wi = inputs.shape
# Hi = 24
# Wi = 24
# N = 1
padding = 0 
Num_rows = 32
Num_Columns = 32           
# dilation = 1         
# stride =2
# Hout = math.floor((Hi + 2*padding - dilation*(Kh-1) -1)/stride + 1)
# Wout = math.floor((Wi + 2*padding - dilation*(Kw-1) -1)/stride + 1)
Hout = Hi + 2*padding - Kh + 1
Wout = Wi + 2*padding - Kw + 1
print(Hout,Wout)

# filters = torch.randn(COUT, CIN, Kh, Kw)


8 8


In [60]:
inputs

tensor([[[[-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
          [-1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1.],
          [-1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
          [-1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
          [-1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.],
          [-1.,  1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1.],
          [ 1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1., -1.],
          [ 1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1.],
          [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.],
          [-1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1., -1.],
          [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.]],

         [[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1.,  

In [61]:
def checkerboard_last_cols(arr: torch.Tensor, C: int) -> torch.Tensor:
    n, m = arr.shape
    if C == 0:
        return arr.clone()  # return a copy even if unchanged
    if C > m:
        raise ValueError("C cannot be larger than the number of columns m.")
    
    arr_copy = arr.clone()
    rows = torch.arange(n).view(-1, 1)                
    cols = torch.arange(m - C, m).view(1, -1)         

    pattern = (rows + cols) % 2
    arr_copy[:, -C:] = pattern
    return arr_copy
# Example
A = torch.ones((6, 8), dtype=torch.int)
A=checkerboard_last_cols(A, C=2)
print(A)


tensor([[1, 1, 1, 1, 1, 1, 0, 1],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 0, 1],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 0, 1],
        [1, 1, 1, 1, 1, 1, 1, 0]], dtype=torch.int32)


In [62]:
def compliment(x):
    x = x.clone()
    neg = -1*x
    pos = x

    pos[pos==-1] = 0
    neg[neg==-1] = 0
    return pos, neg

In [63]:
def map_conv2d(x,w,padding=0):
    _N_, _CIN_, _Hi_, _Wi_ = x.shape
    _COUT_, _CIN_, _Kh_, _Kw_ = w.shape
    pos_w, neg_w = compliment(w)
    mapped_w = torch.empty((_COUT_,_CIN_,2*(_Kh_*_Kw_)))

    pos_w = pos_w.reshape(_COUT_,_CIN_,-1)
    neg_w = neg_w.reshape(_COUT_,_CIN_,-1)
    kernel_size = _Kh_*_Kw_
    for i in range(kernel_size):

        mapped_w[:,:,2*i] = pos_w[:,:,i]
        mapped_w[:,:,2*i+1] = neg_w[:,:,i]
    
    _Hout_ = _Hi_ + 2*padding - _Kh_ + 1
    _Wout_ = _Wi_ + 2*padding - _Kw_ + 1
    
    mapped_x = torch.empty((_N_,_Hout_,_Wout_,_CIN_,2*kernel_size))
    pos_x, neg_x = compliment(x)
    
    for ii in range(_Hout_):
        for jj in range(_Wout_):
            _pos_ = pos_x[:,:,ii:ii+_Kh_,jj:jj+_Kw_]
            _neg_ = neg_x[:,:,ii:ii+_Kh_,jj:jj+_Kw_]
            # print(_pos_.shape)
            _pos_ = _pos_.reshape(_N_,_CIN_,-1)
            _neg_ = _neg_.reshape(_N_,_CIN_,-1)
            one_kernel = torch.empty(_N_,_CIN_,2*kernel_size)
            for z in range(kernel_size):
                one_kernel[:,:,2*z]=_pos_[:,:,z]
                one_kernel[:,:,2*z+1]=_neg_[:,:,z]
            mapped_x[:,ii,jj,:,:] = one_kernel
    return mapped_x,mapped_w

In [64]:
new_inputs,new_weights = map_conv2d(inputs,filters)

In [65]:
def get_conv2d_output(I,Kh, Kw, CIN):
    out = 2*I - Kh*Kw*CIN
    return out

In [66]:
ref = F.conv2d(inputs, filters, padding=0)
ref.shape

torch.Size([1, 16, 8, 8])

In [67]:
out_2d = torch.empty((N,COUT,Hout,Wout))
out_2d.shape

torch.Size([1, 16, 8, 8])

In [68]:
new_weights.shape

torch.Size([16, 6, 50])

In [69]:
out_2d = torch.empty((N,COUT,Hout,Wout))
# print(out_2d)
for ii in range(Hout):
    for jj in range(Wout):
        one_pixel = new_inputs[:,ii,jj,:,:]
        # print(one_pixel.shape)
        out_1d = F.conv1d(one_pixel,new_weights)
        out_1d = out_1d.squeeze(-1)
        # print(out_1d.shape)
        out_2d[:,:,ii,jj] = out_1d
out_2d = get_conv2d_output(out_2d,Kh,Kw,CIN)
torch.equal(out_2d,ref)

True

In [70]:
# def torch_conv_2d(crossbar_inputs,crossbar_weights,_COUT_,Total_dim):
#     _N_,_HOUT_,_WOUT_,crossbar_y,Num_rows = crossbar_inputs.shape
#     _,crossbar_x,_,Num_Columns = crossbar_weights.shape
#     output_conv_2d = torch.zeros(_N_,_COUT_,_HOUT_,_WOUT_)

#     columns_per_crossbar = math.floor(_COUT_/crossbar_x)

#     for ii in range(_HOUT_):
#         for jj in range(_WOUT_):
#             mac_out_columns = torch.zeros((_N_,_COUT_))
#             for cy in range(crossbar_y):
#                 for cx in range(crossbar_x):
#                     tmp_x=crossbar_inputs[:,ii,jj,cy,:]
#                     tmp_w=crossbar_weights[cy,cx,:,:]
#                     checkerboard_last_cols(tmp_w,Num_Columns-columns_per_crossbar)
#                     # tmp_w=crossbar_weights
#                     # print(tmp_x.shape)
#                     # print(tmp_w.shape)
#                     # _out_ = torch.matmul(tmp_w,tmp_x)
#                     _out_ = torch.matmul(tmp_x,tmp_w)

#                     column_start_idx = cx*columns_per_crossbar
#                     if cx==crossbar_x-1:
#                         column_end_idx=Total_dim
#                     else:
#                         column_end_idx = (cx+1)*columns_per_crossbar
                    
#                     mac_out_columns[:,column_start_idx:column_end_idx] += _out_[:,:columns_per_crossbar]
#             output_conv_2d[:,:,ii,jj]=mac_out_columns
#     return output_conv_2d


In [71]:
def run_tile(ii,jj,cx, tmp_x_np, tmp_w_np, Num_rows, Num_Columns, columns_per_crossbar, Total_dim, mode,transient):
    # tmp_w_np = checkerboard_last_cols(tmp_w_np, tmp_w_np.shape[1] - columns_per_crossbar)
    
    _, _out_np = _task(((ii,jj), tmp_x_np), tmp_w_np, Num_rows, Num_Columns, mode, transient)

    column_start_idx = cx * columns_per_crossbar
    column_end_idx = (cx + 1) * columns_per_crossbar
    # column_end_idx = Total_dim if tmp_w_np.shape[1] - columns_per_crossbar < columns_per_crossbar else (cx + 1) * columns_per_crossbar
    return ((ii, jj, column_start_idx, column_end_idx), _out_np[:, :columns_per_crossbar])

def cim_conv_2d(crossbar_inputs, crossbar_weights, _COUT_, mode, Total_dim, transient,max_workers=None):
    _N_, _HOUT_, _WOUT_, crossbar_y, Num_rows = crossbar_inputs.shape
    _, crossbar_x, _, Num_Columns = crossbar_weights.shape
    output_conv_2d = torch.zeros(_N_, _COUT_, _HOUT_, _WOUT_)

    columns_per_crossbar = math.floor(_COUT_ / crossbar_x)

    # Preconvert weights to NumPy once (this is reused for each (ii, jj))
    # weights_np = [[crossbar_weights[cy, cx].numpy() for cx in range(crossbar_x)] for cy in range(crossbar_y)]

    tasks = []
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        for ii in range(_HOUT_):
            for jj in range(_WOUT_):

                for cy in range(crossbar_y):
                    for cx in range(crossbar_x):
                        tmp_x = crossbar_inputs[:, ii, jj, cy, :].numpy()
                        tmp_w = crossbar_weights[cy][cx]
                        # tmp_w = weights_np[cy][cx]
                        tasks.append((ii,jj, cx, tmp_x, tmp_w, Num_rows, Num_Columns, columns_per_crossbar, Total_dim, mode, transient))

        futures = [executor.submit(run_tile, *t) for t in tasks]
        for f in tqdm(as_completed(futures),total=len(tasks),disable=True):
            (ii,jj, col_start, col_end), out_np = f.result()
            out = torch.from_numpy(out_np)
            output_conv_2d[:, col_start:col_end, ii, jj] += out

    return output_conv_2d

In [72]:
def _conv2d_tile_(x,w,Num_rows,Num_Columns,mode,max_workers,transient=False):
    _N_,_HOUT_, _WOUT_,_CIN_, _kernel_size_ = x.shape
    _COUT_, _CIN_, _kernel_size_ = w.shape
    output_conv_2d = torch.zeros(_N_,_COUT_,_HOUT_,_WOUT_)

    whole_input_size = _CIN_*_kernel_size_
    print(f"total inputs: {whole_input_size}")
    crossbar_y = math.ceil((_CIN_*_kernel_size_)/Num_rows)
    crossbar_x = math.ceil(_COUT_/Num_Columns)

    crossbar_weights = torch.zeros((crossbar_y,crossbar_x,Num_rows,Num_Columns))
    crossbar_inputs = torch.zeros((_N_,_HOUT_,_WOUT_,crossbar_y,Num_rows))

    rows_per_crossbar = math.floor(whole_input_size/crossbar_y)

    flatten_x = x.reshape(*x.shape[:-2],-1)
    print(flatten_x.shape)
    flatten_w = w.reshape(*w.shape[:-2],-1).T
    print(flatten_w.shape)
    print(crossbar_inputs.shape)
    print(f"weights shape: {crossbar_weights.shape}")
    print(f"inputs shape: {crossbar_inputs.shape}")

    columns_per_crossbar = math.floor(_COUT_/crossbar_x)

    for ii in range(crossbar_y):
        row_start_idx = ii*rows_per_crossbar
        # if ii==crossbar_y-1:
        #     row_end_idx = flatten_x.shape[-1]
        # else:
        row_end_idx = (ii+1)*rows_per_crossbar
        # print(start_idx,end_idx)
        crossbar_inputs[:,:,:,ii,:rows_per_crossbar] = flatten_x[:,:,:,row_start_idx:row_end_idx]
    # for ii in range(0,whole_input_size,step=inpu)

        for jj in range(crossbar_x):
            column_start_idx = jj*columns_per_crossbar
            # if jj==crossbar_x-1:
            #     column_end_idx=flatten_w.shape[-1]
            # else:
            column_end_idx = (jj+1)*columns_per_crossbar
            
            crossbar_weights[ii,jj,:rows_per_crossbar,:columns_per_crossbar] = flatten_w[row_start_idx:row_end_idx,column_start_idx:column_end_idx]


    # for n in range(_N_):
    output_conv_2d = cim_conv_2d(crossbar_inputs,crossbar_weights,_COUT_,mode,flatten_w.shape[-1],transient=transient,max_workers=max_workers)
    # output_conv_2d = torch_conv_2d(crossbar_inputs,crossbar_weights,_COUT_,flatten_w.shape[-1])
    return output_conv_2d


In [73]:
def conv2d_tile(x,w,Num_rows,Num_Columns,mode,max_workers,transient=False):
    _N_, _CIN_, _Hi_, _Wi_ = x.shape
    _COUT_, _CIN_, _Kh_, _Kw_ = w.shape
    mapped_x, mapped_w = map_conv2d(x,w)
    output_conv_2d = _conv2d_tile_(mapped_x,mapped_w,Num_rows,Num_Columns,mode,max_workers,transient)
    output_conv_2d = get_conv2d_output(output_conv_2d,_Kh_,_Kw_,_CIN_)
    
    return output_conv_2d


In [75]:
def con2d(x,w,Num_rows,Num_Columns,mode,max_workers,transient=False):
    padding = 0
    _N_, _CIN_, _Hi_, _Wi_ = x.shape
    _COUT_, _CIN_, _Kh_, _Kw_ = w.shape
    _HOUT_ = _Hi_ + 2*padding - _Kh_ + 1
    _WOUT_ = _Wi_ + 2*padding - _Kw_ + 1
    output_conv_2d = torch.zeros(_N_,_COUT_,_HOUT_,_WOUT_)
    for idx in range(_N_):
        tmp_x = x[idx].unsqueeze(0)
        output_conv_2d[idx,:,:,:] = conv2d_tile(tmp_x,w,Num_rows,Num_Columns,mode,max_workers,transient=False)
    return output_conv_2d


In [ ]:
output_conv_2d = con2d(inputs,filters,Num_rows,Num_Columns,mode="ideal",max_workers=20,transient=False)
torch.equal(output_conv_2d,ref)

total inputs: 300
torch.Size([1, 8, 8, 300])
torch.Size([300, 16])
torch.Size([1, 8, 8, 10, 32])
weights shape: torch.Size([10, 1, 32, 32])
inputs shape: torch.Size([1, 8, 8, 10, 32])


True

In [ ]:
output_conv_2d = con2d(inputs,filters,Num_rows,Num_Columns,mode="cs",max_workers=20,transient=False)
torch.equal(output_conv_2d,ref)

In [ ]:
output_conv_2d = con2d(inputs,filters,Num_rows,Num_Columns,mode="gs",max_workers=20,transient=False)
torch.equal(output_conv_2d,ref)
ref-output_conv_2d

In [ ]:
fc_weight = model.fc1.weight.data.detach()
fc_weight_b = binarized(fc_weight)
fc_weight_b.shape


torch.Size([120, 256])

In [114]:
fc_input = model.pool2(model.htanh2(model.bn2(model.conv2(x))))
fc_input_b = binarized(fc_input)
fc_input_b = fc_input_b.view(fc_input_b.size(0),-1)
fc_input_b = fc_input_b.detach()
fc_input_b.shape

torch.Size([1, 256])

In [115]:
def map_fc(x,w):
    print(x.shape)
    print(w.shape)
    M, N = w.shape
    _N_, M = x.shape
    pos_w, neg_w = compliment(w) 
    pos_x, neg_x = compliment(x)

    mapped_w = torch.empty((2*M,N))
    mapped_x = torch.empty((_N_,2*M))

    for idx in range(M):
        _pos_w_ = pos_w[idx]
        _neg_w_ = neg_w[idx]
        mapped_w[2*idx]=_pos_w_
        mapped_w[2*idx+1]=_neg_w_

        _pos_x_ = pos_x[:,idx]
        _neg_x_ = neg_x[:,idx]

        mapped_x[:,2*idx] = _pos_x_
        mapped_x[:,2*idx+1] = _neg_x_

    return mapped_x,mapped_w

In [95]:
def get_fc_output(I, M):
    out = 2*I - M
    return out

In [117]:
fc_weight_b.shape

torch.Size([120, 256])

In [103]:
fc_input_b.shape

torch.Size([1, 256])

In [126]:
new_fc_inputs, new_fc_weights = map_fc(fc_input_b,fc_weight_b.T)
fc_py = F.linear(new_fc_inputs,new_fc_weights.T)
fc_py = get_fc_output(fc_py,fc_input_b.shape[-1])
print(fc_py.shape)
ref_fc = F.linear(fc_input_b,fc_weight_b)

torch.equal(ref_fc,fc_py)

torch.Size([1, 256])
torch.Size([256, 120])
torch.Size([1, 120])


True

In [109]:
def  fc_linear(crossbar_inputs,crossbar_weights,N,mode,max_workers,transient):
    crossbar_y,crossbar_x,Num_rows,Num_Columns = crossbar_weights.shape
    _N_, crossbar_y, Num_rows = crossbar_inputs.shape
    output = torch.zeros(_N_,N)
    columns_per_crossbar = math.floor(N/crossbar_x)
    
    tasks = []

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        for ii in range(crossbar_y):
            for jj in range(crossbar_x):
                column_start_idx = jj*columns_per_crossbar
                column_end_idx = (jj+1)*columns_per_crossbar
                tmp_x = crossbar_inputs[:,ii]
                tmp_w = crossbar_weights[ii][jj]
                tmp_w=checkerboard_last_cols(tmp_w,Num_Columns-columns_per_crossbar)
                # out_matmul = torch.matmul(tmp_x,tmp_w)
                args = (((ii,jj),tmp_x),tmp_w,Num_rows,Num_Columns,mode,transient)
                tasks.append(args)
        futures = [executor.submit(_task, *t) for t in tasks]
        for f in tqdm(as_completed(futures),total=len(tasks)):
            (ii,jj), out_matmul = f.result()
            column_start_idx = jj*columns_per_crossbar
            column_end_idx = (jj+1)*columns_per_crossbar
            out_matmul = torch.from_numpy(out_matmul)
            output[:,column_start_idx:column_end_idx] += out_matmul[:,:columns_per_crossbar]

    return output

In [110]:
def _fc_tile_(x,w, Num_rows,Num_Columns,mode,max_workers,transient):
    _N_ , M = x.shape
    M , N = w.shape

    crossbar_y = math.ceil(M/Num_rows)
    crossbar_x = math.ceil(N/Num_Columns)
    # print("crossbar grid",crossbar_y,crossbar_x)

    crossbar_inputs = torch.zeros((_N_,crossbar_y,Num_rows))
    crossbar_weights = torch.zeros((crossbar_y,crossbar_x,Num_rows,Num_Columns))

    rows_per_crossbar = math.floor(M/crossbar_y)
    columns_per_crossbar = math.floor(N/crossbar_x)

    # print(rows_per_crossbar, columns_per_crossbar)

    for ii in range(crossbar_y):
        row_start_idx = ii*rows_per_crossbar
        # if ii==crossbar_y-1:
        #     row_end_idx = M
        # else:
        row_end_idx = (ii+1)*rows_per_crossbar
        crossbar_inputs[:,ii,:rows_per_crossbar] = x[:,row_start_idx:row_end_idx]
    # for ii in range(0,whole_input_size,step=inpu)
        # print(row_start_idx,row_end_idx)

        for jj in range(crossbar_x):
            column_start_idx = jj*columns_per_crossbar
            # if jj==crossbar_x-1:
            #     column_end_idx=N
            # else:
            column_end_idx = (jj+1)*columns_per_crossbar
            # print(column_start_idx,column_end_idx)
            
            crossbar_weights[ii,jj,:rows_per_crossbar,:columns_per_crossbar] = w[row_start_idx:row_end_idx,column_start_idx:column_end_idx]
        
    # print(f"crossbar weigths : {crossbar_weights.shape}")
    # print(f"crossbar inputs : {crossbar_inputs.shape}")
    return fc_linear(crossbar_inputs,crossbar_weights,N,mode,max_workers,transient)

In [111]:
def fc_tile(x,w, Num_rows,Num_Columns,mode,max_workers,transient):
    # w = w.T
    M , N = w.shape
    _N_ , M = x.shape
    mapped_x, mapped_w = map_fc(x,w)
    output_fc = _fc_tile_(mapped_x,mapped_w,Num_rows,Num_Columns,mode,max_workers,transient)
    output_fc = get_fc_output(output_fc,M)
    return output_fc

def fc(x,w, Num_rows,Num_Columns,mode,max_workers,transient):
    # w = w.T
    print(w.shape)
    M , N = w.shape
    _N_ , M = x.shape
    output_fc = torch.empty((_N_,N))

    for idx in range(_N_):
        tmp_x = x[idx].unsqueeze(0)
        output_fc[idx,:] = fc_tile(tmp_x,w, Num_rows,Num_Columns,mode,max_workers,transient)
    return output_fc

In [121]:
output_fc = fc(fc_input_b,fc_weight_b.T,Num_rows,Num_Columns,"ideal",max_workers=8,transient=False)
torch.equal(output_fc,ref_fc)

torch.Size([256, 120])
torch.Size([1, 256])
torch.Size([256, 120])


100%|██████████| 64/64 [00:00<00:00, 119.43it/s]


True

In [123]:
output_fc = fc(fc_input_b,fc_weight_b.T,Num_rows,Num_Columns,"cs",max_workers=8,transient=False)
print(torch.equal(output_fc,ref_fc))
output_fc-ref_fc

torch.Size([256, 120])
torch.Size([1, 256])
torch.Size([256, 120])


100%|██████████| 64/64 [00:00<00:00, 96.27it/s] 


False


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         2., 0., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [124]:
output_fc = fc(fc_input_b,fc_weight_b.T,Num_rows,Num_Columns,"gs",max_workers=8,transient=False)
print(torch.equal(output_fc,ref_fc))
output_fc-ref_fc

torch.Size([256, 120])
torch.Size([1, 256])
torch.Size([256, 120])


100%|██████████| 64/64 [00:00<00:00, 94.12it/s] 


False


tensor([[30., 32., 28., 30., 28., 22., 18., 14.,  4.,  4.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0., -2., -2., -2., -2., -6., -8., -4., -2.,
          0., -2., 32., 32., 32., 30., 28., 24., 12., 10.,  6.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -2., -2., -8., -2., -2.,
          0.,  0.,  0., -8., 32., 32., 30., 32., 22., 22., 24.,  6.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -2.,
          0., -6.,  0., -2.,  0.,  0., 32., 32., 32., 32., 26., 18., 18., 12.,
          0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -8.,  0.,
         -2.,  0.,  0., -4., -6., -4., -4., -2.]])